# Conversational Interface - Chatbot with Claude LLM

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

In this notebook, we will build a chatbot using the Foundation Models (FMs) in Amazon Bedrock. For our use-case we use Claude V3 Sonnet as our foundation models.  For more details refer to [Documentation](https://aws.amazon.com/bedrock/claude/). The ideal balance between intelligence and speed—particularly for enterprise workloads. It excels at complex reasoning, nuanced content creation, scientific queries, math, and coding. Data teams can use Sonnet for RAG, as well as search and retrieval across vast amounts of information while sales teams can leverage Sonnet for product recommendations, forecasting, and targeted marketing. 

## Overview

Conversational interfaces such as chatbots and virtual assistants can be used to enhance the user experience for your customers.Chatbots uses natural language processing (NLP) and machine learning algorithms to understand and respond to user queries. Chatbots can be used in a variety of applications, such as customer service, sales, and e-commerce, to provide quick and efficient responses to users. They can be accessed through various channels such as websites, social media platforms, and messaging apps.


## Chatbot using Amazon Bedrock

![Amazon Bedrock - Conversational Interface](./images/chatbot_bedrock.png)


## Use Cases

1. **Chatbot (Basic)** - Zero Shot chatbot with a FM model
2. **Chatbot using prompt** - template(Langchain) - Chatbot with some context provided in the prompt template
3. **Chatbot with persona** - Chatbot with defined roles. i.e. Career Coach and Human interactions
4. **Contextual-aware chatbot** - Passing in context through an external file by generating embeddings.

## Langchain framework for building Chatbot with Amazon Bedrock
In Conversational interfaces such as chatbots, it is highly important to remember previous interactions, both at a short term but also at a long term level.

LangChain provides memory components in two forms. First, LangChain provides helper utilities for managing and manipulating previous chat messages. These are designed to be modular and useful regardless of how they are used. Secondly, LangChain provides easy ways to incorporate these utilities into chains.
It allows us to easily define and interact with different types of abstractions, which make it easy to build powerful chatbots.

## Building Chatbot with Context - Key Elements

The first process in a building a contextual-aware chatbot is to **generate embeddings** for the context. Typically, you will have an ingestion process which will run through your embedding model and generate the embeddings which will be stored in a sort of a vector store. In this example we are using Titan Embeddings model for this

![Embeddings](./images/embeddings_lang.png)

Second process is the user request orchestration , interaction,  invoking and returing the results

![Chatbot](./images/chatbot_lang.png)

## Architecture [Context Aware Chatbot]
![4](./images/context-aware-chatbot.png)


## Setup

⚠️ ⚠️ ⚠️ Before running this notebook, ensure you've run the [Bedrock boto3 setup notebook](../00_Prerequisites/bedrock_basics.ipynb) notebook. ⚠️ ⚠️ ⚠️ Then run these installs below

**please note**

for we are tracking an annoying warning when using the RunnableWithMessageHistory [Runnable History Issue]('https://github.com/langchain-ai/langchain-aws/issues/150'). Please ignore the warning mesages for now


In [ ]:
%pip install -U langchain-community==0.2.11
%pip install -U --no-cache-dir  \
    "langchain>=0.2.12" \
    sqlalchemy -U \
    "faiss-cpu>=1.7,<2" \
    "pypdf>=3.8,<4" \
    pinecone-client>=5.0.1 \
    tiktoken>=0.7.0 \
    "ipywidgets>=7,<8" \
    matplotlib>=3.9.0 \
    anthropic>=0.32.0 \
    "langchain-aws>=0.1.15"
#%pip install -U --no-cache-dir transformers
#%pip install -U --no-cache-dir boto3


In [ ]:
import warnings

from io import StringIO
import sys
import textwrap
import os
from typing import Optional

# External Dependencies:
import boto3
from botocore.config import Config

warnings.filterwarnings("ignore")


def print_ww(*args, width: int = 100, **kwargs):
    """Like print(), but wraps output to `width` characters (default 100)"""
    buffer = StringIO()
    try:
        _stdout = sys.stdout
        sys.stdout = buffer
        print(*args, **kwargs)
        output = buffer.getvalue()
    finally:
        sys.stdout = _stdout
    for line in output.splitlines():
        print("\n".join(textwrap.wrap(line, width=width)))


def get_bedrock_client(
    assumed_role: Optional[str] = None,
    region: Optional[str] = None,
    runtime: Optional[bool] = True,
):
    """Create a boto3 client for Amazon Bedrock, with optional configuration overrides

    Parameters
    ----------
    assumed_role :
        Optional ARN of an AWS IAM role to assume for calling the Bedrock service. If not
        specified, the current active credentials will be used.
    region :
        Optional name of the AWS Region in which the service should be called (e.g. "us-east-1").
        If not specified, AWS_REGION or AWS_DEFAULT_REGION environment variable will be used.
    runtime :
        Optional choice of getting different client to perform operations with the Amazon Bedrock service.
    """
    if region is None:
        target_region = os.environ.get(
            "AWS_REGION", os.environ.get("AWS_DEFAULT_REGION")
        )
    else:
        target_region = region

    print(f"Create new client\n  Using region: {target_region}")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end="")
        sts = session.client("sts")
        response = sts.assume_role(
            RoleArn=str(assumed_role), RoleSessionName="langchain-llm-1"
        )
        print(" ... successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"][
            "SecretAccessKey"
        ]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]

    if runtime:
        service_name = "bedrock-runtime"
    else:
        service_name = "bedrock"

    bedrock_client = session.client(
        service_name=service_name, config=retry_config, **client_kwargs
    )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client

In [ ]:
import json
import os
import sys

import boto3


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."


boto3_bedrock = get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region="us-west-2",  # os.environ.get("AWS_DEFAULT_REGION", None)
)

## Chatbot (Basic - without context)

We use [CoversationChain](https://python.langchain.com/en/latest/modules/models/llms/integrations/bedrock.html?highlight=ConversationChain#using-in-a-conversation-chain) from LangChain to start the conversation. We also use the [ConversationBufferMemory](https://python.langchain.com/en/latest/modules/memory/types/buffer.html) for storing the messages. We can also get the history as a list of messages (this is very useful in a chat model).

Chatbots needs to remember the previous interactions. Conversational memory allows us to do that. There are several ways that we can implement conversational memory. In the context of LangChain, they are all built on top of the ConversationChain.

**Note:** The model outputs are non-deterministic

In [ ]:
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"  # "anthropic.claude-v2"

messages = [
    {
        "role": "user",
        "content": [{"type": "text", "text": "What is quantum mechanics? "}],
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": "It is a branch of physics that describes how matter and energy interact with discrete energy values ",
            }
        ],
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Can you explain a bit more about discrete energies?",
            }
        ],
    },
]
body = json.dumps(
    {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 100,
        "messages": messages,
        "temperature": 0.5,
        "top_p": 0.9,
    }
)

response = boto3_bedrock.invoke_model(body=body, modelId=modelId)
response_body = json.loads(response.get("body").read())
print(response_body)


def test_sample_claude_invoke(prompt_str, boto3_bedrock):
    modelId = "anthropic.claude-3-sonnet-20240229-v1:0"  # "anthropic.claude-v2"
    messages = [{"role": "user", "content": [{"type": "text", "text": prompt_str}]}]
    body = json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 100,
            "messages": messages,
            "temperature": 0.5,
            "top_p": 0.9,
        }
    )
    response = boto3_bedrock.invoke_model(body=body, modelId=modelId)
    response_body = json.loads(response.get("body").read())
    return response_body


test_sample_claude_invoke("what is quantum mechanics", boto3_bedrock)

#### Introduction to ChatBedrock

**Supports the following**
1. Multiple Models from Bedrock 
2. Converse API
3. Ability to do tool binding
4. Ability to plug with LangGraph flows

In [ ]:
from langchain_aws.chat_models.bedrock import ChatBedrock
from langchain_core.messages import HumanMessage
from langchain_core.messages import HumanMessage, SystemMessage

model_parameter = {"temperature": 0.0, "top_p": 0.5, "max_tokens_to_sample": 2000}
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"  # "anthropic.claude-v2"
bedrock_llm = ChatBedrock(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs=model_parameter,
    beta_use_converse_api=True,
)

messages = [HumanMessage(content="what is the weather like in Seattle WA")]
bedrock_llm.invoke(messages)

#### Due to the converse api flag -- this class corectly formulates the messages correctly

so we can directly use the string mesages

In [ ]:
bedrock_llm.invoke("what is the weather like in Seattle WA?")

#### Ask a follow on

because we have not plugged in any History or context or api's the model wil not be able to answer the question

In [ ]:
bedrock_llm.invoke("is it warm in summers?")

### Ask the same question Meta Llama models

**please make sure you have the models enabled**

In [ ]:
from langchain_aws.chat_models.bedrock import ChatBedrock
from langchain_core.messages import HumanMessage
from langchain_core.messages import HumanMessage, SystemMessage

model_parameter = {"temperature": 0.0, "top_p": 0.5, "max_tokens_to_sample": 2000}
modelId = "meta.llama3-8b-instruct-v1:0"
bedrock_llm = ChatBedrock(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs=model_parameter,
    beta_use_converse_api=True,
)

messages = [HumanMessage(content="what is the weather like in Seattle WA")]
bedrock_llm.invoke(messages)

#### Simple Conversation chain 

**Uses the In memory Chat Message History**

The above example uses the same history for all sessions. The example below shows how to use a different chat history for each session.

**Note**
1. `Chat History` is a variable is a place holder in the prompt template. which will have Human/Ai alternative messages
2. Human query is the final question as `Input` variable
3. config is the `{"configurable": {'session_id_variable':'value,....other keys}` These are passed into the any and all Runnable and wrappers of runnable
4. `RunnableWithMessageHistory` is the class which we wrap the `chain` in to run with history. which is in [Docs link]('https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html#')
5. For production use cases, you will want to use a persistent implementation of chat message history, such as `RedisChatMessageHistory`.
6. This class needs a DICT as a input

Wrap the rag_chain with RunnableWithMessageHistory to automatically handle chat history:

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_aws.chat_models.bedrock import ChatBedrock

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a pirate. Answer the following questions as best you can."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

history = InMemoryChatMessageHistory()


def get_history():
    return history


model_parameter = {"temperature": 0.0, "top_p": 0.5, "max_tokens_to_sample": 2000}
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"  # "anthropic.claude-v2"
chatbedrock_llm = ChatBedrock(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs=model_parameter,
    beta_use_converse_api=True,
)

chain = prompt | chatbedrock_llm | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key="chat_history",
)
memory.chat_memory.add_ai_message("I am a career coach and give career advice")
cl_llm = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0", client=boto3_bedrock
)  # - anthropic.claude-3-sonnet-20240229-v1:0,     anthropic.claude-v2
conversation = ConversationChain(llm=cl_llm, verbose=True, memory=memory)

print_ww(wrapped_chain.invoke({"input": "what is the weather like in Seattle WA?"}))

print(
    "\n\n Now we run The example below shows how to use a different chat history for each session."
)

#### Use the multiple session id's with in memory conversations

In [ ]:
### This below LEVARAGES the In-memory with multiple sessions and session id
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    # print(session_id)
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


chain = prompt | chatbedrock_llm | StrOutputParser()

wrapped_chain = RunnableWithMessageHistory(
    chain,
    get_session_history,
    history_messages_key="chat_history",
)

print_ww(
    wrapped_chain.invoke(
        {"input": "what is the weather like in Seattle WA"},
        config={"configurable": {"session_id": "abc123"}},
    )
)

print(
    "\n\n now ask another question and we will see the History conversation was maintained"
)
print_ww(
    wrapped_chain.invoke(
        {"input": "Ok what are benefits of this weather in 100 words?"},
        config={"configurable": {"session_id": "abc123"}},
    )
)

#### Now we do a Conversation Chat Chain with History and add a Retriever to that convo


[Docs links]('https://python.langchain.com/v0.2/docs/versions/migrating_chains/conversation_retrieval_chain/')

**Chat History needs to be a list since this is message api so alternate with human and user**

1. The ConversationalRetrievalChain was an all-in one way that combined retrieval-augmented generation with chat history, allowing you to "chat with" your documents.

2. Advantages of switching to the LCEL implementation are similar to the RetrievalQA section above:

3. Clearer internals. The ConversationalRetrievalChain chain hides an entire question rephrasing step which dereferences the initial query against the chat history.
4. This means the class contains two sets of configurable prompts, LLMs, etc.
5. More easily return source documents.
6. Support for runnable methods like streaming and async operations.

**Below are the key classes to be used**

1. We create a QA Chain using the qa_chain as `create_stuff_documents_chain(chatbedrock_llm, qa_prompt)`
2. Then we create the Retrieval History chain using the `create_retrieval_chain(history_aware_retriever, qa_chain)`
3. Retriever is wrapped in as `create_history_aware_retriever`
4. `{context}` goes as System prompts which goes into the Prompt templates
5. `Chat History` goes in the Prompt templates like "placeholder", "{chat_history}")

The LCEL implementation exposes the internals of what's happening around retrieving, formatting documents, and passing them through a prompt to the LLM, but it is more verbose. You can customize and wrap this composition logic in a helper function, or use the higher-level `create_retrieval_chain` and `create_stuff_documents_chain` helper method:

#### FAISS as VectorStore

In order to be able to use embeddings for search, we need a store that can efficiently perform vector similarity searches. In this notebook we use FAISS, which is an in memory store. For permanently store vectors, one can use pgVector, Pinecone or Chroma.

The langchain VectorStore API's are available [here](https://python.langchain.com/en/harrison-docs-refactor-3-24/reference/modules/vectorstore.html)

To know more about the FAISS vector store please refer to this [document](https://arxiv.org/pdf/1702.08734.pdf).

#### Titan embeddings Model

Embeddings are a way to represent words, phrases or any other discrete items as vectors in a continuous vector space. This allows machine learning models to perform mathematical operations on these representations and capture semantic relationships between them.

Embeddings are for example used for the RAG [document search capability](https://labelbox.com/blog/how-vector-similarity-search-works/) 


In [ ]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.vectorstores import FAISS

from langchain.embeddings import BedrockEmbeddings

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)

s3_path = "s3://jumpstart-cache-prod-us-east-2/training-datasets/Amazon_SageMaker_FAQs/Amazon_SageMaker_FAQs.csv"
!aws s3 cp $s3_path ./rag_data/Amazon_SageMaker_FAQs.csv

loader = CSVLoader("./rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars, each row consists in a question column and an answer column
documents_aws = loader.load() #
print(f"Number of documents={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Number of documents after split and chunking={len(docs)}")
vectorstore_faiss_aws = None

    
vectorstore_faiss_aws = FAISS.from_documents(
    documents=docs,
     embedding = br_embeddings
)

print(f"vectorstore_faiss_aws: number of elements in the index={vectorstore_faiss_aws.index.ntotal}::")



#### First we do the simple Retrieval QA chain -- No chat history but with retriver
[Docs link]('https://python.langchain.com/v0.2/docs/versions/migrating_chains/retrieval_qa/')

Key points
1. The chain in QA uses the variable as the first value, can be input or question  and so the prompt template for the Human query has to have the `Question` or `input` as the variable
2. This chain will re formulate the question, call the retriver and then answer the question
3. Our prompt template removes any answer where retriver is not needed and so no answer is obtained
4. Context goes into the system prompts section

In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

condense_question_system_template = """
    You are an assistant for question-answering tasks. ONLY Use the following pieces of retrieved context to answer the question.
    If the answer is not in the context below , just say you do not have enough context. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.
    Context: {context} 
    """


condense_question_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", condense_question_system_template),
        (
            "human",
            "{input}",
        ),  # expected by the qa chain as it sends in question as the variable
    ]
)

model_parameter = {"temperature": 0.0, "top_p": 0.5, "max_tokens_to_sample": 2000}
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"  # "anthropic.claude-v2"
chatbedrock_llm = ChatBedrock(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs=model_parameter,
    beta_use_converse_api=True,
)


def format_docs(docs):
    # print(docs)
    return "\n\n".join(doc.page_content for doc in docs)


qa_chain = (
    {
        "context": vectorstore_faiss_aws.as_retriever()
        | format_docs,  # can work even without the format
        "input": RunnablePassthrough(),
    }
    | condense_question_prompt
    | chatbedrock_llm
    | StrOutputParser()
)

print_ww(
    qa_chain.invoke(input="What are autonomous agents?")
)  # cannot be a dict object here)

### Ask the same question to Meta Models
**Note with the converse API we do not need to formulate or change any prompts**

In [ ]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

condense_question_system_template = """
    You are an assistant for question-answering tasks. ONLY Use the following pieces of retrieved context to answer the question.
    If the answer is not in the context below , just say you do not have enough context. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.
    Context: {context} 
    """


condense_question_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", condense_question_system_template),
        (
            "human",
            "{input}",
        ),  # expected by the qa chain as it sends in question as the variable
    ]
)

model_parameter = {"temperature": 0.0, "top_p": 0.5, "max_tokens_to_sample": 2000}
modelId = "meta.llama3-8b-instruct-v1:0"
chatbedrock_llm = ChatBedrock(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs=model_parameter,
    beta_use_converse_api=True,
)


def format_docs(docs):
    # print(docs)
    return "\n\n".join(doc.page_content for doc in docs)


qa_chain = (
    {
        "context": vectorstore_faiss_aws.as_retriever()
        | format_docs,  # can work even without the format
        "input": RunnablePassthrough(),
    }
    | condense_question_prompt
    | chatbedrock_llm
    | StrOutputParser()
)

print_ww(
    qa_chain.invoke(input="What are autonomous agents?")
)  # cannot be a dict object here)

#### Now we get a real answer as we invoke where retriever gives context

Use the Helper method to create the Retiever QA Chain

In [ ]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

condense_question_system_template = """
    You are an assistant for question-answering tasks. ONLY Use the following pieces of retrieved context to answer the question.
    If the answer is not in the context below , just say you do not have enough context. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.
    Context: {context} 
    """

condense_question_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", condense_question_system_template),
        ("human", "{input}"),
    ]
)
model_parameter = {"temperature": 0.0, "top_p": 0.5, "max_tokens_to_sample": 2000}
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"  # "anthropic.claude-v2"
chatbedrock_llm = ChatBedrock(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs=model_parameter,
    beta_use_converse_api=True,
)
qa_chain = create_stuff_documents_chain(chatbedrock_llm, condense_question_prompt)

convo_qa_chain = create_retrieval_chain(vectorstore_faiss_aws.as_retriever(), qa_chain)

print_ww(
    convo_qa_chain.invoke(
        {
            "input": "What are the options for model explainability in SageMaker?",
            "config": {"configurable": {"session_id": "abc123"}},
        }
    )
)  # cannot be a dict object here)

#### Now we create Chat Conversation which has history and retrieval context
So we use the HISTORY AWARE Retriever and create a chain

1. We create a stuff chain
2. Then we pass it to the create retrieval chain method -- we could have used the LCEL as well to create the chain

In [ ]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

contextualized_question_system_template = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualized_question_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualized_question_system_template),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    chatbedrock_llm,
    vectorstore_faiss_aws.as_retriever(),
    contextualized_question_prompt,
)

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("placeholder", "{contex}"),
        ("placeholder", "{chat_history}"),
        ("human", "Explain this  {input}."),
    ]
)

qa_chain = create_stuff_documents_chain(chatbedrock_llm, qa_prompt)

convo_qa_chain = create_retrieval_chain(history_aware_retriever, qa_chain)

convo_qa_chain.invoke(
    {
        "input": "What are the options for model explainability in SageMaker?",
        "chat_history": [],
    }
)

#### Auto add the history to the Chat with Retriever

Wrap with Runnable Chat History with Session id and run the chat conversation

![Amazon Bedrock - Conversational Interface](./images/context_aware_history_retriever.png)

borrowed from https://github.com/langchain-ai/langchain

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

model_parameter = {"temperature": 0.0, "top_p": 0.5, "max_tokens_to_sample": 2000}
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"  # "anthropic.claude-v2"
chatbedrock_llm = ChatBedrock(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs=model_parameter,
    beta_use_converse_api=True,
)

contextualized_question_system_template = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualized_question_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualized_question_system_template),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    chatbedrock_llm,
    vectorstore_faiss_aws.as_retriever(),
    contextualized_question_prompt,
)


qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If the answer is not present in the context, just say you do not have enough context to answer. \
If the input is not present in the context, just say you do not have enough context to answer. \
If the question is not present in the context, just say you do not have enough context to answer. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(chatbedrock_llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

# - Wrap the rag_chain with RunnableWithMessageHistory to automatically handle chat history:
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    # print(session_id)
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


chain_with_history = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
result = chain_with_history.invoke(
    {"input": "What kind of bias can SageMaker detect?"},
    config={"configurable": {"session_id": "session_1"}},
)
result

### As a follow on question

1. The phrase `it` will be converted based on the chat history
2. Retriever gets invoked to get relevant content based on chat history 

In [ ]:
follow_up_result = chain_with_history.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "session_1"}},
)
print_ww(follow_up_result)

In [ ]:
follow_up_result = chain_with_history.invoke(
    {"input": "Will it help?"}, config={"configurable": {"session_id": "session_1"}}
)
print_ww(follow_up_result)

#### Now ask a random question

In [ ]:
follow_up_result = chain_with_history.invoke(
    {"input": "Give me a few tips on how to plant a  new garden."},
    config={"configurable": {"session_id": "session_1"}},
)
follow_up_result

Let's see how the semantic search works:
1. First we calculate the embeddings vector for the query, and
2. then we use this vector to do a similarity search on the store

In [ ]:
v = br_embeddings.embed_query("R in SageMaker")
print(v[0:10])
results = vectorstore_faiss_aws.similarity_search_by_vector(v, k=4)
for r in results:
    print_ww(r.page_content)
    print("----")

#### Memory
In any chatbot we will need a QA Chain with various options which are customized by the use case. But in a chatbot we will always need to keep the history of the conversation so the model can take it into consideration to provide the answer. In this example we use the [ConversationalRetrievalChain](https://python.langchain.com/docs/modules/chains/popular/chat_vector_db) from LangChain, together with a ConversationBufferMemory to keep the history of the conversation.

Source: https://python.langchain.com/docs/modules/chains/popular/chat_vector_db

Set `verbose` to `True` to see all the what is going on behind the scenes.

In [ ]:
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

print_ww(CONDENSE_QUESTION_PROMPT.template)

#### Parameters used for ConversationRetrievalChain
* **retriever**: We used `VectorStoreRetriever`, which is backed by a `VectorStore`. To retrieve text, there are two search types you can choose: `"similarity"` or `"mmr"`. `search_type="similarity"` uses similarity search in the retriever object where it selects text chunk vectors that are most similar to the question vector.

* **memory**: Memory Chain to store the history 

* **condense_question_prompt**: Given a question from the user, we use the previous conversation and that question to make up a standalone question

* **chain_type**: If the chat history is long and doesn't fit the context you use this parameter and the options are `stuff`, `refine`, `map_reduce`, `map-rerank`

If the question asked is outside the scope of context, then the model will reply it doesn't know the answer

**Note**: if you are curious how the chain works, uncomment the `verbose=True` line.

#### Do some prompt engineering

You can "tune" your prompt to get more or less verbose answers. For example, try to change the number of sentences, or remove that instruction all-together. You might also need to change the number of `max_tokens` (eg 1000 or 2000) to get the full answer.

### In this demo we used Claude V3 sonnet LLM to create conversational interface with following patterns:

1. Chatbot (Basic - without context)

2. Chatbot using prompt template(Langchain)

3. Chatbot with personas

4. Chatbot with context